In [2]:
import requests
from bs4 import BeautifulSoup
import time

從csv讀取飯店名稱跟url

In [27]:
import pandas as pd
df = pd.read_csv('../0806/tapei_tripadvisor_top300.csv')
all_hotel = df['title'].to_list()
all_uri = df['uri'].to_list()
# 讀取現有的使用者url
userDF = pd.read_csv('../0813/tapei_tripadvisor_985user.csv')
users_uri = userDF['profile_uri'].to_list()
users_names = userDF['username'].to_list()

In [3]:
user_example = 'https://www.tripadvisor.com.tw/Profile/IQ943?tab=reviews'

In [28]:
# 抓使用者的fb連結
def get_user_profile(user_list):
    data_list = []
    for i, url in enumerate(user_list):
        time.sleep(2)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,'html.parser')
        user_fb = soup.select('.social-member-MemberInfo__member_info--S4DCO')
        # 評論過多的可能要用seleniumn的lazy loading處理
        user_comment = soup.select('.social-sections-POICarousel__gutter--w8x-S.social-sections-POICarousel__first--1aVoT.social-sections-POICarousel__last--1wXdF')
       
        hotel_list = []
        data = {'userName': users_names[i],
                'user_link': url}
        
        # 要被click才會顯示，所以要用seleniumn
        # print(user_fb[0].a.get('href'))
        for j, hotel in enumerate(user_comment):
            # 只顯示餐廳
            if 'Hotel' in hotel.div.a.get('href'):
                hotel_list.append('https://www.tripadvisor.com.tw' + hotel.div.a.get('href'))
        data['commented_hotel'] = ', '.join(hotel_list)
        data_list.append(data)
    
    return data_list
            

In [30]:
all_data = get_user_profile(users_uri)

In [31]:
import pandas as pd
# 用 list存多個 dict然後轉成 df
data_df = pd.DataFrame.from_dict(all_data)
data_df.head()

,userName,user_link,commented_hotel
0,brooke,https://www.tripadvisor.com.tw/Profile/brookeggbb,https://www.tripadvisor.com.tw/Hotel_Review-g1...
1,橘子妮,https://www.tripadvisor.com.tw/Profile/Orange0630,
2,Ting,https://www.tripadvisor.com.tw/Profile/Hjadey,https://www.tripadvisor.com.tw/Hotel_Review-g1...
3,Melody,https://www.tripadvisor.com.tw/Profile/Wuchij,
4,jotsw,https://www.tripadvisor.com.tw/Profile/yrhhi,https://www.tripadvisor.com.tw/Hotel_Review-g1...


In [32]:
data_df.to_csv('./tripadvisor_user_hotel_comment.csv', index=False, encoding='utf_8_sig')